# Re-producing the Shift data
from the study "Flying in 2050"

## Hypothesis

In [1]:
import numpy as np
import configparser

%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use("./ipynb_inline.mplstyle")

from aviation import traj_objective, traj_ref, optimisation_traj

In [2]:
config = configparser.ConfigParser()
configfilename = "./aviation_parameters.ini"
config.read(configfilename)

['./aviation_parameters.ini']

In [3]:
begin_year = float(config["InitialState"]["begin_year"])
emission_2018_fr = float(config["InitialState"]["emission_2018_fr"])
emission_2018_wd = float(config["InitialState"]["emission_2018_wd"])
budget_carbone_2018_50_fr = float(config["InitialState"]["budget_carbone_2018_50_fr"])
budget_carbone_2018_50_wd = float(config["InitialState"]["budget_carbone_2018_50_wd"])
years = np.arange(begin_year, 2051, 1. )

In [4]:

trajectory_ref_word = traj_ref(config, years)
        
trajectory_obj_word = traj_objective(config, years)


In [10]:
from scipy.interpolate import make_interp_spline, BSpline

# 300 represents number of points to make between T.min and T.max
yearsnew = np.linspace(years.min(), years.max(), 300) 

optimisation_shortterm_operation = float(config["simpleModel"]["optimisation_shortterm_operation"])
optimisation_technologique = float(config["simpleModel"]["optimisation_technologique"])

spl = make_interp_spline(years, trajectory_ref_word, k=3, bc_type=([(1, 0.0)], [(2, 0.0)]))  # type: BSpline

traj_operation_wd = optimisation_traj(trajectory_ref_word, optimisation_shortterm_operation)
traj_techno_wd = optimisation_traj(traj_operation_wd, optimisation_technologique)

spl_op = make_interp_spline(years, traj_operation_wd, k=3, bc_type=([(1, 0.0)], [(2, 0.0)]))  # type: BSpline
spl_tech = make_interp_spline(years, traj_techno_wd, k=3, bc_type=([(1, 0.0)], [(2, 0.0)]))  # type: BSpline


0.003355918720319928
0.02078540275399865


# Making a Bokeh plot

In [11]:
from bokeh.io import output_notebook, output_file

from bokeh.layouts import gridplot, layout, Spacer
from bokeh.layouts import column, row
from bokeh.models import CustomJS, Slider
from bokeh.plotting import ColumnDataSource, figure, output_file, show, save


output_notebook()

Loading BokehJS ...

In [12]:

x = yearsnew
ref_traj = spl(yearsnew)
op_traj = spl_op(yearsnew)
tech_traj = spl_tech(yearsnew)
obj_spl = make_interp_spline(years,trajectory_obj_word , k=3, bc_type=([(1, 0.0)], [(2, 0.0)]))  # type: BSpline

objectif_traj = obj_spl(yearsnew)

source = ColumnDataSource(data=dict(x=x, ref_traj=ref_traj, oper_trj=op_traj, tech_traj=tech_traj, objectif_traj=objectif_traj))

plot = figure( plot_width=850,
              plot_height=400,
              toolbar_location="below",
           toolbar_sticky=False,
              tools= "crosshair,pan,wheel_zoom,box_zoom,save,reset",
             background_fill_color="#fafafa")

plot.line('x', 'ref_traj',
          source=source,
          line_width=3,
          line_alpha=0.9,
          line_color="red", 
          legend_label="Reference +4%/yr")

plot.line('x', 'objectif_traj',
          source=source,
          line_width=3,
          line_alpha=0.6,
          line_color="black", 
          line_dash="dashed",
          legend_label="Objectif -3.39%/yr")

plot.line('x', 'oper_trj',
          source=source,
          line_width=3,
          line_alpha=0.6,
          legend_label="optimisation Operation")

plot.line('x', 'tech_traj',
          source=source,
          line_width=3,
          line_alpha=0.6,
          line_color="green",
          legend_label="optimisation Technologique")

opt_slider = Slider(start=0, end=100, value=50, step=1, title="Total Technological Optimisation factor in 2050 (%) ")
op_slider = Slider(start=0, end=30, value=10, step=1, title="Total Operation Optimisation factor in 2050 (%) ")
freq_slider = Slider(start=0.1, end=10, value=1, step=.1, title="Frequency")


callback = CustomJS(args=dict(source=source,
                              opt_op=op_slider,
                              opt_tech=opt_slider,
                             ),
                    code="""
    const data = source.data;
    const tech_percent = opt_tech.value;
    const op_perc = opt_op.value;

    const x = data['x']
    const tech_traj = data['tech_traj']
    const ref_traj = data['ref_traj']
    const oper_trj = data['oper_trj']
    
    const factor_op = 1-(1-op_perc/100) ** (1/ (x.length) )
    const factor_tech = 1-(1-tech_percent/100) ** (1/ (x.length) )
    
    for (var i = 0; i < x.length; i++) {
        oper_trj[i] = ref_traj[i] * (1 - factor_op) ** (i) ;
    }
    
    for (var i = 0; i < x.length; i++) {
        tech_traj[i] = oper_trj[i] * (1 - factor_tech) ** (i) ;
    }
    
    source.change.emit();
""")

opt_slider.js_on_change('value', callback)
op_slider.js_on_change('value', callback)


plot.legend.location = "top_left"
plot.legend.click_policy="mute"

plot.title.text = 'Evolution of the World total emission'
plot.xaxis.axis_label = "year"
plot.yaxis.axis_label = "Emission [MtCO2]"


layout = column( [ row([column(opt_slider, op_slider, ), Spacer() ]), 
                  plot
              ],sizing_mode='fixed' 
)


#output_file("transport_interactive_evolution.html", title="transport_evolution")



In [13]:
show(layout)

In [37]:
from bokeh.resources import CDN
from bokeh.plotting import figure
from bokeh.embed import autoload_static

folderpath = "/home/tavant/Documents/blogs/myblog/static/"
scriptfilename = "/js/shift_aviation_2050.js"
divfilename = "/js/shift_aviation_2050.html"


js, tag = autoload_static(layout, CDN, script_path=scriptfilename)

with open(folderpath+scriptfilename, "w") as f:
    f.write(js)
    
print(tag)

with open(folderpath+divfilename, "w") as fp:
    fp.write(tag)
      


<script src="/js/shift_aviation_2050.js" id="5459a3f1-5ee0-45a6-a0c3-4f996e38ef2b"></script>
